In [1]:
import torch
import numpy as np
import pandas as pd
import gc

from peft import LoraConfig, get_peft_model, TaskType
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments,
    Trainer,
)

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
tokenizer = AutoTokenizer.from_pretrained("IlyaGusev/mbart_ru_sum_gazeta")
model = AutoModelForSeq2SeqLM.from_pretrained("IlyaGusev/mbart_ru_sum_gazeta")


In [4]:
train_df = pd.read_csv('C:/repos/gosling-sight/data/train_data.csv', sep=',')
train_df

,text,labels
0,"Во-вторых, адаптивный режим, который автоматич...","Ультраширик обновили с 12 до 13 Мп, но убрали ..."
1,Основной модуль — передовой Sony IMX800 на 54 ...,\nОсновной модуль — передовой Sony IMX800. Ему...
2,"А чтобы запечатлеть динамичную сцену днём, выс...",Кадры получаются чёткими и резкими по всей пло...
3,"Хорошая резкость, но не совсем точная цветопер...","Хорошая резкость, но не совсем точная цветопер..."
4,В кадре можно разместить целую компаниюТак выг...,"Качество на высоте, хотя снимали в помещении. ..."
5,"Тёмная область сверху зашумлена, но освещённые...","Приложение камеры функционирует шустро, быстро..."
6,Радуют широкий динамический диапазон и умеренн...,Радуют широкий динамический диапазон и умеренн...
7,"Причём в лучшую сторону: усилится контраст, ка...","\nУсилится контраст, кадр станет более объёмны..."
8,Прошлые модели линейки фотографировали заметно...,Прошлые модели линейки фотографировали заметно...
9,У основной камеры и телевика одинаковое разреш...,У основной камеры и телевика одинаковое разреш...


In [14]:
dataset = Dataset.from_pandas(train_df)


In [19]:
def preprocess_function(sample):
    inputs = tokenizer(
        sample["text"], padding=True, truncation=True, return_tensors="pt"
    )
    labels = tokenizer(
        sample["labels"], padding=True, truncation=True, return_tensors="pt"
    )
    
    inputs["labels"] = labels["input_ids"]
    
    return inputs

In [20]:
dataset = dataset.map(preprocess_function, batched=True)  # processing dataset

  0%|          | 0/1 [00:00<?, ?ba/s]

In [21]:
lora_config = LoraConfig(
    r=16, 
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)
model = get_peft_model(model, lora_config)

In [22]:
training_args = Seq2SeqTrainingArguments(
    output_dir="../models/mbart_fine_tune/",
    learning_rate=1e-3, # higher learning rate
    num_train_epochs=5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    weight_decay=0.24,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset.shuffle(seed=16),
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [23]:
torch.cuda.empty_cache()
gc.collect()

2776

In [25]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `PeftModelForSeq2SeqLM.forward` and have been ignored: text. If text are not expected by `PeftModelForSeq2SeqLM.forward`,  you can safely ignore this message.
c:\Users\al_po\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 48
  Num Epochs = 5
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 120
  Number of trainable parameters = 2359296


  0%|          | 0/120 [00:00<?, ?it/s]



Training completed. Do not forget to share your model on huggingface.co/models =)




{'train_runtime': 812.118, 'train_samples_per_second': 0.296, 'train_steps_per_second': 0.148, 'train_loss': 8.263887532552083, 'epoch': 5.0}


TrainOutput(global_step=120, training_loss=8.263887532552083, metrics={'train_runtime': 812.118, 'train_samples_per_second': 0.296, 'train_steps_per_second': 0.148, 'train_loss': 8.263887532552083, 'epoch': 5.0})

In [26]:
trainer.save_model("../models/mbart_fine_tune/v1")

Saving model checkpoint to ../models/mbart_fine_tune/v1
Trainer.model is not a `PreTrainedModel`, only saving its state dict.


In [60]:
%%capture
model.eval()

In [33]:
def generate(tokenizer, model, article_text):
    input_ids = tokenizer(
    [article_text],
    max_length=1200,
    add_special_tokens=True,
    padding=True,
    truncation=True,
    return_tensors="pt"
    )["input_ids"].to(device)

    output_ids = model.generate(
        input_ids=input_ids,
        no_repeat_ngram_size=6,
        
    
    )[0]

    summary = tokenizer.decode(output_ids, skip_special_tokens=True)
    return summary

In [31]:
text = "Во-вторых, адаптивный режим, который автоматически обрезает и стабилизирует изображение, ориентируясь на интенсивность тряски. Ультраширик обновили с 12 до 13 Мп, но убрали автофокус. Фронталку заменили с 12 на 32 Мп, однако и тут куда-то делся автофокус. В опциях смартфона есть несколько функций, продлевающих срок службы батареи. Такого от малыша не ожидаешь При аккумуляторе ёмкостью 4300 мАч особых чудес автономности не ждёшь. Ещё можно установить ограничение на зарядку — восполнять энергию только до 80 или 90 процентов. Настройки предлагают пять пресетов, регулирующих цветовую температуру, так что каждый сумеет добиться идеальной для себя картинки. За пределами гейминга экран ZenFone 10 позволяет выбрать между 60, 90, 120 Гц и автоматической регулировкой. При 60 и 90 Гц телефон блокирует заданное значение и не снижает даже во время бездействия. В красном цвете покрытие напоминает наждачную бумагу, как раньше делала OnePlus. В настройках представлены два варианта интерфейса: один максимально близок к чистой ОС Android, а другой воплотил фирменное видение бренда и предлагает больше возможностей по кастомизации. ASUS обещает только 2 крупных обновления Android и 4 года патчей безопасности, тогда как OnePlus и Samsung предлагают 4 версии ОС и 5 лет поддержки. Фото: nextpit В обычном режиме система под длительной нагрузкой уменьшает быстродействие до 60% от своего максимума. И они даже слегка уменьшились по сравнению с прошлогодним ZenFone 9, в результате чего звук стал менее объёмным и громким. Качество аудио в целом высокое, хотя и уступает Galaxy S23 или iPhone 14. Во-первых, есть исключительно оптическая стабилизация, полагающаяся на аппаратную составляющую."

In [34]:
generate(tokenizer, model, text)

c:\Users\al_po\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\generation\utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'В опциях смартфона есть несколько функций, продлевающих срок службы батареи. Такого от малыша не ожидаешь При аккумуляторе ёмкостью 4300 мАч особых чудес автономности не ждёшь. Качество аудио в целом высокое, хотя и уступает Galaxy S23 или iPhone 14. В обычном режиме система под длительной нагрузкой уменьшает быстродействие до 60% от своего максимума. И они даже слегка уменьшились по сравнению с прошлогодним ZenFone 9, в результате чего звук стал менее объёмным и громким. ASUS обещает только 2 крупных обновления Android и 4 года патчей безопасности, тогда как OnePlus и Samsung предлагают 4 версии ОС и 5 лет поддержки.'

In [35]:
df = pd.read_csv('C:/repos/gosling-sight/data/sum_table.csv', sep=',')

In [38]:
df['cat_sum'] = df['join_cat'].apply(lambda x: generate(tokenizer, model, x))


c:\Users\al_po\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\generation\utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [39]:
df['abs_sum'] = df['conclusion'].apply(lambda x: generate(tokenizer, model, x))

c:\Users\al_po\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\generation\utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [137]:
print(df['abs_sum'].values[1])
print(df['conclusion'].values[1])
print(df['sum_conclusion'].values[1])

В остальном перед нами сбалансированный аппарат с интересным дизайном, производительным железом, топовым экраном и неплохой автономностью. Итоги Honor 70 — интересный смартфон среднего ценового сегмента с хорошими фотовозможностями, благодаря алгоритмам, заимствованным от флагманов вендора и качественным дисплеем. Но, хотелось бы ещё увидеть стереодинамики и хотя бы минимальную брызгозащиту.
 Итоги Honor 70 — интересный смартфон среднего ценового сегмента с хорошими фотовозможностями, благодаря алгоритмам, заимствованным от флагманов вендора и качественным дисплеем. Но, хотелось бы ещё увидеть стереодинамики и хотя бы минимальную брызгозащиту. В остальном перед нами сбалансированный аппарат с интересным дизайном, производительным железом, топовым экраном и неплохой автономностью.
 Итоги Honor 70 — интересный смартфон среднего ценового сегмента с хорошими фотовозможностями, благодаря алгоритмам, заимствованным от флагманов вендора и качественным дисплеем. Но, хотелось бы ещё увидеть сте

In [133]:
df.to_csv('../data/sum_table.csv', encoding='utf-8', index=False)

In [68]:
df['abs_sum'][1:5].values

array(['В остальном перед нами сбалансированный аппарат с интересным дизайном, производительным железом, топовым экраном и неплохой автономностью. Итоги Honor 70 — интересный смартфон среднего ценового сегмента с хорошими фотовозможностями, благодаря алгоритмам, заимствованным от флагманов вендора и качественным дисплеем. Но, хотелось бы ещё увидеть стереодинамики и хотя бы минимальную брызгозащиту.',
       'По фотосъёмке этот аппарат сейчас на вершине мобильного Олимпа. Большой вырез на экране? Не ради прихоти, а для размещения одной из самых продвинутых систем разблокировки — работает она безукоризненно. Камера с изменяемой диафрагмой и оптической стабилизацией, превосходный OLED без ШИМ и с высокой частотой обновления, качественные материалы, быстрая зарядка и флагманский процессор — HUAWEI Mate 50 Pro задаёт действительно высокую планку качества. Смартфон сокращает разрыв между мобильными камерами и фотоаппаратами со сменной оптикой.',
       'Российский Huawei Mate 50 — неоднозна

In [70]:
df['cat_sum'][1:5].values

array(['Главный модуль — передовой Sony IMX800 на 54 мегапикселя с диафрагмой f/1.9 и размером 1/1,49 дюйма. Ему в помощь приставлены сенсор на 50 мегапикселей в паре со сверхширокоугольным объективом на 122 градуса и датчик глубины на 2 Мп. Стойкий аккумулятор с быстрой зарядкой В тонкий корпус Honor 70 производитель умудрился установить АКБ на 4800 мАч. Традиционный для бренда экран-водопад В Honor 70 установлена изогнутая под углом 58 градусов 6,67-дюймовая OLED-матрица. К яркости матрицы и ШИМ вопросов нет — заметить мерцание невооружённым глазом не получится. Матовое покрытие есть только у изумрудно-зелёной версии. На бумаге он может похвастаться новейшей камерой I',
       'А чтобы запечатлеть динамичную сцену днём, выставить более закрытую — кадр получится чётким и резким по всей площади. Модуль на 64 Мп с оптической стабилизацией обеспечивает 3,5-кратное зумирование. Смартфон оснастили 6,74-дюймовой OLED-матрицей с изогнутым стеклом. Производитель заявляет о подавлении ШИМ на в